In [2]:
from elasticsearch import Elasticsearch
HOST = 'elasticsearch-master.projet-ssplab'

def elastic():
    """Connection avec Elastic sur le data lab"""
    es = Elasticsearch([{'host': HOST, 'port': 9200, 'scheme': 'http'}], http_compress=True, request_timeout=200)
    return es

es = elastic()

In [3]:
# Importation des bases
import functions as fc
dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [23]:
df = dict_data["etablissements"]
df = df.rename({'numero_siret': "numero_siret_true"}, axis = 1)

In [25]:
df["adresse"].head()

0                                  Plaine Saint Pierre
1                                 17 avenue de l'adour
2                                      1 route de lens
3                                               BP 104
4    Boulevard Dunant (entrée par Allée des Grands ...
Name: adresse, dtype: object

In [5]:

requete_type = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{ "match": {{ "rs_denom":   "{nom_etablissement}" }}}},
        {{ "match": {{ "geo_adresse": "{adresse}" }}}},
        {{ "match": {{ "sir_adr_et_com_lib": "{commune}" }}}}
      ],
      "minimum_should_match": 2,
      "filter": [
        {{ "match":  {{ "adr_et_post": "{code_postal}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''

In [18]:
df.shape

(8963, 15)

In [22]:
df["adresse"].isna().sum()

0

In [ ]:
df["code_postal"]

In [19]:
# Il est nécessaire de spécifier l'index associé à chaque requête
header = '{"index" : "sirus_2020"}'

multiple_requetes = ""

# On itère sur le dataframe d'établissements polluants pour ajouter une requête spécifique à chacun d'entre eux
n_etab = df.shape[0] # Pour l'exemple, on prend les 10 premiers

for index, row in df.iloc[0:n_etab][['nom_etablissement', 'adresse', 'code_postal', 'commune']].iterrows():
    
    multiple_requetes+= header
    multiple_requetes+= '\n'
    multiple_requetes+= requete_type.format_map(row).replace("\n","")
    multiple_requetes+= '\n'

In [20]:
res = es.msearch(body = multiple_requetes)

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


RequestError: RequestError(400, 'x_content_parse_exception', '[1:149] [bool] failed to parse field [should]')

In [8]:
res['responses'][0]['hits']['hits']

[{'_index': 'sirus_2020_e_3_ngr_bool',
  '_type': '_doc',
  '_id': '81162101000024',
  '_score': 5.0,
  '_source': {'sirus_id': '811621010',
   'nic': '00024',
   'ape': '8010Z',
   'apet': '8010Z',
   'eff_3112_et': '3.0',
   'eff_etp_et': '2.0',
   'eff_et_effet_daaaammjj': '20181231',
   'enseigne_et1': '',
   'nom_comm_et': '',
   'adr_et_loc_geo': '3403200276',
   'adr_et_compl': '',
   'adr_et_voie_num': '15',
   'adr_et_voie_repet': '',
   'adr_et_voie_type': '',
   'adr_et_voie_lib': 'PLAINE SAINT PIERRE',
   'adr_et_cedex': '',
   'adr_et_distsp': '',
   'sir_adr_et_com_lib': 'BEZIERS',
   'adr_et_post': '34500',
   'adr_et_l1': 'ALTEA SECURITE BEZIERS',
   'adr_et_l2': '',
   'adr_et_l3': '',
   'adr_et_l4': '15 PLAINE SAINT PIERRE',
   'adr_et_l5': '',
   'adr_et_l6': '34500 BEZIERS',
   'adr_et_l7': '',
   'nic_siege': '00024',
   'unite_type': '1',
   'region': '76',
   'adr_depcom': '34032',
   'region_impl': '76',
   'region_mult': 'QASI',
   'tr_eff_etp': '01',
   'cj':

In [9]:
# on rappatrie les siret trouvés 
res['responses'][0]['hits']['hits'][0]["_source"]["siret_id"]

'81162101000024'

In [10]:
res['responses'][1]['hits']['hits'][0]["_source"]["siret_id"]

'87760842200011'

In [16]:
[res['responses'][i]['hits']['hits'][0]["_source"]["siret_id"] for i in range(4) if res['responses'][i]['hits']['hits']]

['81162101000024', '87760842200011', '53295706500016']

In [17]:
import numpy as np


['81162101000024', '87760842200011', '53295706500016', nan]

In [16]:
df["siret_elastic"] = [res['responses'][i]['hits']['hits'][0]["_source"]["siret_id"] if res['responses'][i]['hits']['hits'] else np.NaN for i in range(4) ]

,identifiant,nom_etablissement,numero_siret_true,adresse,code_postal,commune,departement,region,coordonnees_x,coordonnees_y,code_epsg,code_ape,libelle_ape,code_eprtr,libelle_eprtr
0,9073402,STEP - BEZIERS,41003460701688,Plaine Saint Pierre,34500,BEZIERS,HERAULT,OCCITANIE,672948.910000,1.813634e+06,27572.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...
1,9076401,STEP - Pont de l'aveugle,20006710600241,17 avenue de l'adour,64600,ANGLET,PYRENEES-ATLANTIQUES,NOUVELLE-AQUITAINE,-1.498014,4.350209e+01,4326.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...
2,9076202,STEP - Loison Sous Lens,57202552600813,1 route de lens,62218,LOISON-SOUS-LENS,PAS-DE-CALAIS,HAUTS-DE-FRANCE,2.856548,5.043594e+01,4326.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...
3,9077801,SIAAP - Site Seine Aval,25755000400044,BP 104,78603,MAISONS-LAFFITTE,YVELINES,ILE-DE-FRANCE,2.140590,4.898410e+01,4326.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...
4,9079001,UDEP-BELFORT,20006905200070,Boulevard Dunant (entrée par Allée des Grands ...,90000,BELFORT,TERRITOIRE-DE-BELFORT,BOURGOGNE-FRANCHE-COMTE,6.861504,4.762541e+01,4326.0,3700Z,Collecte et traitement des eaux usées,5.(f),Installations de traitement des eaux urbaines ...
